In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from sklearn.datasets import load_breast_cancer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import random

In [21]:
# filename = Name of the file with input data
# Retrieves data from CSV file and splits it into train and test sets
def get_data():
    #link to dataset in github repo
    data = pd.read_csv("https://raw.githubusercontent.com/ML-Racer-Performance-Prediction/NASCAR/main/Documentation/NASCAR(1905-2024(March))%201.csv", index_col=0)

    labels = data.index
    data = data.values

    #Split the dataset into input and output segments
    driversY = labels
    driversX = data

    return driversY, driversX
    #Divide into train and test sets and shuffle
    #xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 42)

    #Convert data sets into tensors
    #return (xTrain, xTest, yTrain, yTest)

In [23]:
[driversY, driversX] = get_data()

print(driversX[0])
print(driversY[0])

[0.00000000e+00 0.00000000e+00 6.66666670e-02 9.11577000e-04
 2.81000000e+01 2.65000000e+01 8.00000000e-01 4.66666667e-01]
ZANE SMITH


In [ ]:
# MLP model
# Three layer MLP with relu activation functions
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super().__init__()
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.layer = nn.Linear(in_features = input_size, out_features = hidden_size1)
        self.layer2 = nn.Linear(in_features = hidden_size1, out_features = hidden_size2)
        self.layer3 = nn.Linear(in_features = hidden_size2, out_features = 35)

    def forward(self, x):
        x = self.flatten(x)
        x = self.layer(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        return x.sort()

# Create model with hidden layers of 128 and 64 nodes
MLP = MLP(35*10, 128, 64)
print(MLP)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
  (layer): Linear(in_features=423, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=47, bias=True)
)


In [ ]:
# Establish the loss function and optimizer function to be used
lossFunc = nn.L1Loss()
optimizer = torch.optim.SGD(MLP.parameters(), lr = 0.001, momentum = 0.3)

In [ ]:
# Compute the number of correct values
def correct(output, target):
    predicted = output.argmax(1)                            # pick digit with largest network output
    correct_ones = (predicted == target).type(torch.float)  # 1.0 for correct, 0.0 for incorrect
    return correct_ones.sum().item()

In [ ]:
# num_epochs = number of epochs through which the model was trained
# loss = Cross Entropy loss of the model
def draw_loss(epochCount, loss, name):
    plt.plot(range(epochCount), loss, marker = ".", markersize = 1, label = name)

In [ ]:
# Train algorithm that takes in training DataLoader and the model
def train(trainLoader, model):
    # Iterate through data loader and train for every batch
    for (data, target) in trainLoader:
        output = MLP(data) # Output prediction of MLP
        loss = lossFunc(output, target) # Compute loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [ ]:
# Set the default epoch count to 100
epochCount = 100

# Create lists for plotting data to compare train vs test accuracy and loss
lossList = list()
correctList = list()
testLossList = list()
testCorrectList = list()

# Retrieve data and create loaders for iteration and batch creation
[xTrain, xTest, yTrain, yTest] = get_data()
[trainLoader, testLoader] = data_iter(1, xTrain, yTrain, xTest, yTest)

# Train data for each epoch and store statistics in lists
for i in range(epochCount):
    train(trainLoader, MLP)

    output = MLP(xTrain) # Output prediction of MLP
    loss = lossFunc(output, yTrain) # Compute loss
    lossList.append(loss.detach().numpy())
    correctList.append(correct(output, yTrain)/800)

    output = MLP(xTest) # Output prediction of MLP
    loss = lossFunc(output, yTest) # Compute loss
    testLossList.append(loss.detach().numpy())
    testCorrectList.append(correct(output, yTest)/200)

# Plot results from the model
plt.plot()
plt.title("Cross Entropy Loss vs. Epochs")
plt.xlabel("Epoch")
plt.ylabel("Cross Entropy Loss")
draw_loss(epochCount, lossList, "Train")
draw_loss(epochCount, testLossList, "Test")
plt.legend()
plt.show()

plt.plot()
plt.title("Accuracy vs. Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
draw_loss(epochCount, correctList, "Train")
draw_loss(epochCount, testCorrectList, "Test")
plt.legend()

KeyError: 'Driver'